In [2]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

### **Install Dependencies**

In [ ]:
!pip install -r "/content/drive/MyDrive/mini-project-01/requirements.txt"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 42.8 MB/s eta 0:00:00
   ━━

### **Import Required Libraries**

In [ ]:
from pypdf import PdfReader
import yaml
import os
from dotenv import load_dotenv
from sentence_transformers import CrossEncoder
from langchain_core.documents import Document
from pathlib import Path
from typing import Dict,List
import sys
from weaviate.classes.config import Configure,Property,DataType
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig
from collections import defaultdict
import torch
from datasets import load_dataset
import pandas as pd
import time
import weaviate
import warnings

In [ ]:
##Ignore warnings
warnings.filterwarnings("ignore")

In [ ]:
##current working directory
PROJECT_ROOT=Path.cwd()/'drive/MyDrive/mini-project-01'
os.chdir(PROJECT_ROOT)


In [ ]:
print(f"Current working directory:{PROJECT_ROOT}")

Current working directory:/content/drive/MyDrive/mini-project-01


In [ ]:
## import user define python modules
path=str(PROJECT_ROOT/'src')
sys.path.append(path)

from src.services.clean_text import clean_document
from src.services.llm_services import llm_configuration,text_embedding,get_finetuned_model

### **Load Config File**

In [ ]:
## Load yaml file
def load_yaml(file_path):
    with open(file_path,"r") as file:
        config=yaml.safe_load(file)

    return config

##Load the yaml configuration file
config=load_yaml(str(PROJECT_ROOT/"config.yaml"))


### **LLM,Embedding and Cross-Encoder-Reranker**

In [ ]:
## Lad the finetuned LLama model
finetuned_model=get_finetuned_model(config)

Loading base model.....


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loading finetuned adapter...
Finetuned model ready


In [ ]:
##Initiliaze LLM,Embedding and Cross-Encder
embedding_model=text_embedding(config)
reranker=CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

##print three type of models
print()
print("="*60)
print(f"Fine Tuned LLM:{config['hub_model_name']}")
print(f"Embedding:{config["text_emb_provider"]}/{config["text_emb_mdoel"]}")
print(f"Reranker:cross-encoder/ms-marco-MiniLM-L-6-v2")
print("="*60)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Fine Tuned LLM:uber_finetune_model
Embedding:sbert/sentence-transformers/all-MiniLM-L6-v2
Reranker:cross-encoder/ms-marco-MiniLM-L-6-v2


### **Load Data**

In [ ]:
## Read PDF file and extract text
def read_pdf(data_path):
    reader=PdfReader(data_path)
    text=""
    for page in reader.pages:
        text+=page.extract_text() + "\n"
    return text

##Load the data
pdf_dir=PROJECT_ROOT/'data/raw'
pdf_dir.mkdir(parents=True, exist_ok=True)

pdf_file=list(pdf_dir.glob("*.pdf"))

## Read the PDF file
document=read_pdf(pdf_file[0])

## Number of pages and characters in the document
print("Loading Document...")
print("="*60)
print(f"Number of pages in the document:{len(PdfReader(pdf_file[0]).pages)}")
print(f"Number of characters in the document: {len(document)}")
print("="*60)
print("Document loaded")

print()

## After cleaning the document
print("Cleaning Document...")
print("="*60)
cleaned_document=clean_document([document])
print(f"Number of characters in the cleaned document: {len(cleaned_document[0])}")
print("="*60)
print("Cleaned Document")

Loading Document...
Number of pages in the document:142
Number of characters in the document: 640074
Document loaded

Cleaning Document...
Number of characters in the cleaned document: 151814
Cleaned Document


### **Chunking Strategy**

In [ ]:
def chunk_text(text,chunk_size=config["chunk_size"],overlap=config["overlap_size"],source="Annual Uber Report 2024"):
  chunks=[]
  start=0
  chunk_id=0

  while start<len(text):
    end=start+chunk_size
    chunk_text=text[start:end]

    if chunk_text:
      chunks.append({
          "id":chunk_id,
          "source":source,
          "chunk size":chunk_size,
          "chunk":chunk_text,
      })

      chunk_id+=1

    start+=chunk_size-overlap
  return chunks

print("Chunking the document...")
print("="*60)
chunks=chunk_text(cleaned_document[0])
print(f"Number of chunks: {len(chunks)}")
print("="*60)
print("Document chunked")


Chunking the document...
Number of chunks: 113
Document chunked


### **Weaviate client calling**

In [ ]:
client = weaviate.connect_to_embedded()

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 12752


### **Create a Collection**

In [ ]:
client.collections.create(
    name="AdvanceHybridRAG",
    vectorizer_config=Configure.Vectorizer.none(),  # we provide embeddings
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="source", data_type=DataType.TEXT),
        Property(name="chunk_id", data_type=DataType.INT),
    ],
)

### **Insert chunks**

In [ ]:
collection = client.collections.get("AdvanceHybridRAG")

for chunk in chunks:
    vector = embedding_model.embed_query(chunk["chunk"])

    collection.data.insert(
        properties={
            "text": chunk["chunk"],
            "source": chunk["source"],
            "chunk_id": chunk["id"],
        },
        vector=vector,
    )

### **Dense Retrivel(Weaviate)**

In [ ]:
def dense_retrivel(query:str,embedded_model,top_k=config["similarity_top_k"])->List[Dict]:
  """
   Dense vector-based retrieval using embeddings
  """

  query_vector=embedded_model.embed_query(query)

  response = client.collections.get("AdvanceHybridRAG").query.near_vector(
      near_vector=query_vector,
      limit=top_k,
      return_properties=["text", "source","chunk_id"]
  )
  return [o.properties for o in response.objects]


### **Check Sample Query**

In [ ]:
## pass the chunks documents into the dense_retrivel
result_dense=dense_retrivel(query = "How many trading days must the common stock exceed the threshold?"
,embedded_model=embedding_model,
                      top_k=config["similarity_top_k"])

### **Sparse Retrivel(BM25)**

In [ ]:
def sparse_retrivel(query:str,top_k=config["similarity_top_k"])->List[Dict]:
  """
  Sparse keyword-based retrieval using BM25.
  """
  collection = client.collections.get("HybridRAGLibrarian")

  response = collection.query.bm25(
      query=query,
      limit=top_k,
      return_properties=["text", "source","chunk_id"],
   )

  return [obj.properties for obj in response.objects]

### **Check Sample Query**

In [ ]:
result_sparse= sparse_retrivel(query = "How many trading days must the common stock exceed the threshold?",
                      top_k=config["similarity_top_k"])

In [ ]:
result_dense

[{'text': 's (“ the \nshares of our common stock initially underlyi ng the 2028 Convertible Notes. By entering into the Cappe d Calls, we expect to red uce \nadjustments under the terms of the Capped Ca lls. The Capped Calls were included in a dditional paid-in capital in the consolidat ed \nIn December 2020, we issued $1.15 billion a ggregate principal amount of 0% convertible senior notes due in 2025 (the “2025 \nbusiness day immediately precedi ng September 15, 2025 only under the following circumstances: (i) during any calendar quarter \nprice of our common stock for at least 20 trading days (whether or not consecutive) during a pe riod of 30 consecutive trading d ays \nconversion price on each applicable trading day; (ii) during the five business day period after a ny ten consecutive trading day period \n(the “2025 Convertible Notes measurement period”) in which th e trading price (as defined in the indenture governing 2025 \n \nConvertible Notes) per $1,000 principal amount of no

In [ ]:
result_sparse

[{'text': 'l of our outstanding 2025 \nSenior Note in December 2023. As a result, we recognized an immaterial loss on debt extinguishment for the year ended December \nprice of our common stock for at least 20 trading days (whether or not consecutive) during the period of 30 consecutive trading days \n \nconversion price on each applicable trading day; (ii) during the five business day period after a ny ten consecutive trading day period \nConvertible Notes) per $1,000 principal amount of notes for each trading day of the 2028 Convertible Notes measurement period w as \nbusiness on the second scheduled trading day immediately preceding the maturity date, holders may convert all or any portion of their \nconverted and pay or deliver as the case may be, cash, shares of our common stock, or a combination of cash and shares of our \non or after December 5, 2026, if the last reported sale price of our common stock has been at least 130% of the conversion pric e then \nin effect for at least

### **Hybrid Fusion(Sparse(BM25)+Dense(Weaviate))**

In [ ]:
## get the results from both retrievel process(sparse and dense)
def hybrid_retrivel(query:str,
                    embedding_model,
                    top_k=10):
  dense_result=dense_retrivel(query=query,embedded_model=embedding_model,top_k=top_k)
  sparse_result=sparse_retrivel(query=query,top_k=top_k)

  return dense_result,sparse_result

## apply reciprocal rank fusion(rrf)
def rrf_fusion(dense_results:List[Dict],sparse_results:List[Dict],
               k:int=60,top_k:int=config["similarity_top_k"])->List[Dict]:

  """
   Combine dense and sparse results using Reciprocal Rank Fusion

  """
  scores=defaultdict(float)
  documents={}

  ##dense retrivel for ranking
  for rank,chunk in enumerate(dense_results):
    doc_id=f"{chunk["chunk_id"]}"
    scores[doc_id]+=1/(k+rank+1)
    documents[doc_id]=chunk

  ##sparse retrivel for ranking
  for rank,chunk in enumerate(sparse_results):
    doc_id=f"{chunk["chunk_id"]}"
    scores[doc_id]+=1/(k+rank+1)
    documents[doc_id]=chunk

  ##sort by rrf score
  ranked_docs=sorted(scores.keys(),key=lambda x:scores[x],reverse=True)

  # Attach RRF score to each document
  fused_results = []
  for doc_id in ranked_docs[:top_k]:
      doc = documents[doc_id].copy()
      doc["rrf_score"] = round(scores[doc_id], 3)
      fused_results.append(doc)

  return fused_results

## end to end hybrid retrivel pipeline
def retrive_with_rrf(query,embedding_model,top_k=config["similarity_top_k"]):

  dense_results, sparse_results = hybrid_retrivel(
      query=query,
      embedding_model=embedding_model
   )

  fused_results = rrf_fusion(
      dense_results=dense_results,
      sparse_results=sparse_results,
      top_k=10
  )

  return fused_results

In [ ]:
## call the cuntion
query="How many jurisdictions does the text explicitly mention the company is evaluating for further licenses?"
results = retrive_with_rrf(
    query=query,
    embedding_model=embedding_model,
    top_k=10
)

In [ ]:
print(f"🔀 Hybrid (RRF) Search:{query}")
print()

for i,result in enumerate(results,start=1):
  print(f"[{i}] (RRF:{result['rrf_score']}) {result["text"][:200]}")
  print("="*60)

🔀 Hybrid (RRF) Search:How many jurisdictions does the text explicitly mention the company is evaluating for further licenses?

[1] (RRF:0.046) delivery services, grocery delivery services, and 
network companies, licensing regulations and taxation. These laws and regulations are constantly evolving and may be interprete d, 
improve platform 
[2] (RRF:0.031) er changes to our business model in certain jurisdictions. If, as a result of legislation o r judicial 
decisions, we are required to classify Drivers as employees, we would incur significant addition
[3] (RRF:0.03) portion of our cash flows from operations to pay interest and principal on our 
investments, which may in turn limit our ability to implement our business strategy, heighten our vulnerability to downt
[4] (RRF:0.016) we have been required to change our business model, due 
or permits to operate and must continue to comply with the license or permit requireme nts or risk revocation. In addition, we m ay not 
and re
[5] (R

### **Reranking with Cross Endcoder**

In [ ]:
def rerank_cross_encoder(query:str,results:List[Dict],top_k=config["similarity_top_k"])->List[Dict]:
  """
  Rerank using cross encoder
  """
  ##prepare query,result pairs
  pairs=[[query,result["text"]] for result in results]

  ## calculate score using cross-encoder
  scores=reranker.predict(pairs)

  for i,result in enumerate(results):
    result["rerank_score"]=float(scores[i])

  reranked = sorted(results, key=lambda x: x["rerank_score"], reverse=True)[:top_k]
  return reranked

##call the function and test reranking results
rerank_result=rerank_cross_encoder(query,results)

print(f"Reranked Result:{query}")
print()
for i,result in enumerate(rerank_result,start=1):
  print(f"  [{i}] (rerank: {result['rerank_score']:.3f}) {result['text'][:100]}...")


Reranked Result:How many jurisdictions does the text explicitly mention the company is evaluating for further licenses?

  [1] (rerank: -2.378) delivery services, grocery delivery services, and 
network companies, licensing regulations and taxa...
  [2] (rerank: -6.355) rmore, in certain of these jurisdictions, we 
to scrutiny by government authorities. Our c ontinuati...
  [3] (rerank: -7.278) we have been required to change our business model, due 
or permits to operate and must continue to ...


### **Librarian Part(LLM Generation) -- Fine Tuned Model**

In [ ]:
def build_context(chunks, max_chars=5000):
    """
    Combine top-ranked chunks into a single context string
    """
    context = []
    total_chars = 0

    for chunk in chunks:
        text = chunk["text"].strip()
        if total_chars + len(text) > max_chars:
            break
        context.append(text)
        total_chars += len(text)

    return "\n\n".join(context)


In [ ]:
print(f"Loading Tokenizer:{config["base_model"]}")
tokenizer=AutoTokenizer.from_pretrained(config["base_model"],trust_remote_code=True)

## prompt templete
def prompt_format(user_text,
                  system_prompt="You are a helpful financial analyst. Answer strictly using the provided context.",
                  assistant_text=None,
                  generation_prompt=True):

  ##----Format of CHAT ML------------
  messages=[
      {"role":"system","content":system_prompt},
      {"role":"user","content":user_text}
  ]
  if assistant_text is not None:
    messages.append({"role":"assistant","content":assistant_text})

  return tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=generation_prompt)

Loading Tokenizer:meta-llama/Meta-Llama-3-8B-Instruct


In [ ]:
def query_librarian(question:str,
                    embedding_model,
                    top_k:int=10,
                    system_prompt:str="You are a helpful financial analyst. Answer strictly using the provided context."):

  """
   Final Adance Hybrid RAG pipeline:
   Dense+BM25 ---> RRF --->Corss-Encoder --->Fine Tuned LLM
  """

  ## 1.Hybrid retrival and RRF calcuation
  _fusion_results=retrive_with_rrf(
      query=question,
      embedding_model=embedding_model
      )

  ## 2. Cross-encoder reranking
  _reranked_results=rerank_cross_encoder(
      query=question,
      results=_fusion_results)

  ## 3. Build grounded context
  context=build_context(_reranked_results)

  ## 4. Build the final prompt
  user_prompt=f"""
   Use the following context to answer the question.
   If the answer is not explicitly mention in the context,respond exactly with:
   'The provided context does not contain this information'

   Context:
   {context}

   Question:
   {question}

  """

  formatted_prompt = prompt_format(
        user_text=user_prompt,
        system_prompt=system_prompt
    )


  ## 5. Tokenize
  inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True
    ).to(finetuned_model.device)


  ## 6. Generate anwser
  with torch.no_grad():
      outputs = finetuned_model.generate(
          **inputs,
          max_new_tokens=config["max_tokens"],
          do_sample=config["do_sample"],
          temperature=config["temperature"],##deteministic
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.eos_token_id
      )
  ## 7. Decode
  answer = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

  return answer, _reranked_results


### **Evaluation using ROUGE-L and LLM-as a judge**

In [ ]:
dataset=load_dataset(
    "json",
    data_files={"test":"/content/drive/MyDrive/mini-project-01/data/final/test.jsonl"}
)

## save the dataset as pandas dataframe
test_dataset=pd.DataFrame(dataset["test"])

### **Check the inference time**

In [ ]:
def run_rag(question):
    start = time.time()
    answer, _ = query_librarian(question,embedding_model=embedding_model)
    latency = (time.time() - start) * 1000
    return answer, latency

### **Evaluating using ROUGE-L**

In [ ]:
from rouge_score import rouge_scorer

scorer=rouge_scorer.RougeScorer(['rougeL'],use_stemmer=True)

def rouge_l(prediction, reference):
    return scorer.score(reference, prediction)["rougeL"].fmeasure

In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency= run_rag(row[1]["question"])
  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"Rouge-L Score:{rouge_l(answer,row[1]['anwser']):.4f}")
  print("="*60)

  i+=1

  if i==10:
    break

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:The price of the common stock must exceed the conversion price for at least 20 trading days during a 30 consecutive trading day period.
Latency:4089.59 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
Rouge-L Score:0.7917
Customer Question:Summarize the implications of failing to comply with healthcare-related regulations as outlined in the report.
Assistant Answer:Failing to comply with healthcare-related regulations could result in fines or penalties, which could harm our ability to operate our business.
Latency:3460.36 ms
Reference Answer:Failing to comply with healthcare-related regulations may result in government authorities assessing significant fines.
Rouge-L Score:0.5405
Customer Question:What was th

### **Evaluating using LLM-as a judge**

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

## OpenAI API key
load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")

if not openai_api_key:
  print("Warning:no API key is found in .env file")
  print("Looking for a OpenAI API key....")
  gemini_api_key=input("Enter your OpenAI API key:")

  use_llm_as_judge=False
  judge_model=False

else:
  ## Configure Openai with new API key
  client_ai=OpenAI(api_key=openai_api_key)
  judge_model=client_ai
  model_name="gpt-4o-mini"
  use_llm_as_judge=True

  print(f"OpenAI API key configure with:{model_name}")
  print(f"Using OpenAI package")

OpenAI API key configure with:gpt-4o-mini
Using OpenAI package


In [ ]:
## define the function for llm-as a judge
def llm_as_judge(question,prediction,reference):
  """
   Use a OpenAI model(GPT-4o-mini) as judge to check the quality of the answer
  """
  if not use_llm_as_judge:
    return None,"API key is not available"

  judge_prompt=f"""
  You are an expert evaluator for question-answering AI systems.

INSTRUCTION (User Question):
{question}

REFERENCE ANSWER (Ground Truth):
{reference}

MODEL OUTPUT:
{prediction}

Evaluate the MODEL OUTPUT using the following criteria:

1. **Faithfulness**
   - Is the answer strictly supported by the reference?
   - Does it avoid hallucinations or unsupported claims?

2. **Accuracy**
   - Are the facts, numbers, conditions, and entities correct?
   - Are there any factual errors or misinterpretations?

3. **Completeness**
   - Does the answer cover all key points present in the reference?
   - Are any important details missing?

4. **Clarity & Coherence**
   - Is the answer well-structured and easy to understand?
   - Is the language concise and unambiguous?

5. **Safety & Appropriateness**
   - Does the answer avoid unsafe, misleading, or speculative content?
   - Does it avoid unnecessary medical/financial/legal advice beyond the reference?

---

### SCORING INSTRUCTIONS
- Assign a **single overall score from 1 to 5**, where:
  - **1** = Very poor (incorrect, hallucinated, unsafe)
  - **2** = Poor (major issues)
  - **3** = Fair (partially correct but incomplete or unclear)
  - **4** = Good (mostly correct with minor issues)
  - **5** = Excellent (fully correct, faithful, and clear)

---

### RESPONSE FORMAT (STRICT)
Respond using **exactly** the following format:

SCORE: [1–5]/5
REASONING: [2 concise sentences explaining the score]
  """

  response = judge_model.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert evaluator for question-answering AI systems."},
            {"role": "user", "content": judge_prompt}
        ],
        temperature=0.0,
    )

  return response.choices[0].message.content.strip()


In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency= run_rag(row[1]["question"])
  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"LLM-as judge Score:{llm_as_judge(row[1]["question"],answer,row[1]['anwser'])}")
  print("="*60)

  i+=1
  if i==10:
    break


Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:The price of the common stock must exceed the conversion price for at least 20 trading days during a 30 consecutive trading day period.
Latency:3803.23 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
LLM-as judge Score:SCORE: 5/5  
REASONING: The model output accurately reflects the requirement that the price of the common stock must exceed a certain threshold for at least 20 trading days during a 30-day period, aligning perfectly with the reference answer. It is clear, coherent, and does not contain any unsupported claims or errors.
Customer Question:Summarize the implications of failing to comply with healthcare-related regulations as outlined in the report.
Assistant Answer:Failing to comply with healthca

### **Librarian Part(LLM Generation) -- Base Model**

In [ ]:
## 4-bit Quantization configuration
bnb_config=BitsAndBytesConfig(
    load_in_4bit=config["load_in_4bit"],
    bnb_4bit_compute_dtype=config["bnb_4bit_compute_dtype"],
    bnb_4bit_quant_type=config["bnb_4bit_quant_type"],
    bnb_4bit_use_double_quant=config["bnb_4bit_use_double_quant"]
)

## Load the 4-bit quantized model
print(f"Loading base model:{config["base_model"]}.....")

base_model = AutoModelForCausalLM.from_pretrained(
    config["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto"
)
print("Loaded 4-bit quantized model")

Loading base model:meta-llama/Meta-Llama-3-8B-Instruct.....


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loaded 4-bit quantized model


In [ ]:
def query_librarian(question:str,
                    embedding_model,
                    top_k:int=10,
                    system_prompt:str="You are a helpful financial analyst. Answer strictly using the provided context."):

  """
   Final Adance Hybrid RAG pipeline:
   Dense+BM25 ---> RRF --->Corss-Encoder --->Fine Tuned LLM
  """

  ## 1.Hybrid retrival and RRF calcuation
  _fusion_results=retrive_with_rrf(
      query=question,
      embedding_model=embedding_model
      )

  ## 2. Cross-encoder reranking
  _reranked_results=rerank_cross_encoder(
      query=question,
      results=_fusion_results)

  ## 3. Build grounded context
  context=build_context(_reranked_results)

  ## 4. Build the final prompt
  user_prompt=f"""
   Use the following context to answer the question.

   Context:
   {context}

   Question:
   {question}

  """

  formatted_prompt = prompt_format(
        user_text=user_prompt,
        system_prompt=system_prompt
    )


  ## 5. Tokenize
  inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True
    ).to(base_model.device)


  ## 6. Generate anwser
  with torch.no_grad():
      outputs = base_model.generate(
          **inputs,
          max_new_tokens=config["max_tokens"],
          do_sample=False,
          temperature=None,##deteministic
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.eos_token_id
      )
  ## 7. Decode
  answer = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

  return answer, _reranked_results


### **ROUGE-L Evaluation for Based Model**

In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency= run_rag(row[1]["question"])
  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"Rouge-L Score:{rouge_l(answer,row[1]['anwser']):.4f}")
  print("="*60)

  i+=1

  if i==10:
    break

Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:According to the context, the specific conversion condition is:

"...if the last reported sale price of our common stock has been at least 130% of the conversion price then in effect for at least 20 trading days (whether or not consecutive) during any 30 consecutive trading day period (including the last trading day of such period) ending on, and including, the trading day immediately preceding the date on which we provide notice of conversion..."

So, the answer is: 20 trading days.
Latency:7122.96 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
Rouge-L Score:0.3048
Customer Question:Summarize the implications of failing to comply with healthcare-related regulations as outlined in the report.
Assistant Answ

### **LLM as a judge Evaluation for Based Model**

In [ ]:
i=0
for row in test_dataset.iterrows():
  answer,latency= run_rag(row[1]["question"])
  print(f"Customer Question:{row[1]["question"]}")
  print(f"Assistant Answer:{answer}")
  print(f"Latency:{latency:.2f} ms")
  print(f"Reference Answer:{row[1]['anwser']}")
  print(f"LLM-as judge Score:{llm_as_judge(row[1]["question"],answer,row[1]['anwser'])}")
  print("="*60)

  i+=1
  if i==10:
    break


Customer Question:How many trading days must the price of the common stock exceed for at least 20 days during a 30-day period for a specific conversion condition?
Assistant Answer:According to the context, the specific conversion condition is:

"...if the last reported sale price of our common stock has been at least 130% of the conversion price then in effect for at least 20 trading days (whether or not consecutive) during any 30 consecutive trading day period (including the last trading day of such period) ending on, and including, the trading day immediately preceding the date on which we provide notice of conversion..."

So, the answer is: 20 trading days.
Latency:7083.89 ms
Reference Answer:The price of the common stock must exceed for at least 20 trading days during a 30-day period for a specific conversion condition.
LLM-as judge Score:SCORE: 5/5  
REASONING: The model output accurately reflects the requirement that the price of the common stock must exceed for at least 20 tradi